# Metastable manifold in a three spin dissipative quantum east model
In this notebook we demonstrate how the general code provided in the metastability module can be used to construct the simplex of an open quantum system: in this case, that of a dissipative quantum east model defined externally. This properties and dynamics of this simplex are then briefly analysed.

First we need to import a necessary packages and source code.

In [1]:
import sys
import os
import time
import numpy as np
source_path = os.path.join("source/")
sys.path.insert(0,source_path)
import metastability
source_path = os.path.join("models/")
sys.path.insert(0,source_path)
import dissipative_quantum_east as dqe
from scipy import linalg

Next we define the model, at parameters known to exhibit metastability, with a gap between the 4th and 5th eigenmodes.

In [2]:
sites = 3
decay_rate = 1
field = 0.1
temperature = 0.0001
hardness = 0.999
model = dqe.master_operator(sites, decay_rate, field, temperature, hardness)

Now we need to diagonalize the master operator: this is done by representing the Lindbladian as a matrix, then transforming the resulting eigenvectors into their corresponding matrix representation. Note the large gap between the 4th and 5th eigenvalues.

In [3]:
metastable_phases = sites + 1
extra_eigenvalues = 1
eigenvalues, left_eigenmatrices, right_eigenmatrices = model.spectrum(metastable_phases, 
                                                                      extra_eigenvalues)
print("Eigenvalues:")
print(eigenvalues)
new_left_eigenmatrix_1 = (left_eigenmatrices[2] + left_eigenmatrices[3])/2
new_left_eigenmatrix_2 = (left_eigenmatrices[2] - left_eigenmatrices[3])/2j
new_right_eigenmatrix_1 = (right_eigenmatrices[2] + right_eigenmatrices[3])
new_right_eigenmatrix_2 = (right_eigenmatrices[2] - right_eigenmatrices[3])/1j
left_eigenmatrices[2] = new_left_eigenmatrix_1
left_eigenmatrices[3] = new_left_eigenmatrix_2
right_eigenmatrices[2] = new_right_eigenmatrix_1
right_eigenmatrices[3] = new_right_eigenmatrix_2

Eigenvalues:
[ 0.00000000e+00-0.j -9.32500000e-07-0.j -2.96070000e-06-0.j
 -2.96070000e-06+0.j -2.07537518e-02-0.j]


In [4]:
simplex_vertices = metastability.simplex_vertices(left_eigenmatrices[::-1], 
                                                  rotations = 0)
probability_operators = metastability.probability_operators(left_eigenmatrices[::-1],
                                                            simplex_vertices)
classicality = metastability.classicality(probability_operators)

print("Vertices:")
for i in range(metastable_phases):
	print(np.around(simplex_vertices[i].real, 3))
    
print("\nClassicality:")
print(classicality.real)

Vertices:
[ 0.485 -0.317 -1.149  1.   ]
[ 0.025  0.577 -1.149  1.   ]
[0.    0.    0.005 1.   ]
[-0.51  -0.26  -1.149  1.   ]

Classicality:
5.365805361150655e-07
